## Preprocessing

In [1]:
# Run this code if the machine doesn't have tensorflow ready
!pip install tensorflow

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://raw.githubusercontent.com/Alphaomegainfinity/deep-learning-Alphabet-Soup/main/Resources/charity_data.csv")
application_df.head()

2023-01-10 06:05:13.530603: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-10 06:05:16.753993: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-10 06:05:16.754033: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-10 06:05:23.811230: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df = application_df.drop(columns =['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.

application_df[application_df.dtypes.index.tolist()].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning

application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

count_freq = dict(application_df['APPLICATION_TYPE'].value_counts())
application_types_to_replace = dict(filter(lambda i:i[1]<528, count_freq.items()))

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning

application_df["CLASSIFICATION"].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1

application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = application_df[application_df['CLASSIFICATION'].isin(application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()<1883].index)]['CLASSIFICATION']
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`

application_convert = pd.get_dummies(application_df)
application_convert

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays

y = application_convert["IS_SUCCESSFUL"]
X_application_convert = application_convert.drop(["IS_SUCCESSFUL"], axis=1)

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X_application_convert, y, random_state = 42, stratify =y) 

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Checking the number of feature:

number_input_feature = X_train.shape[1]
number_input_feature

43

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense (units = 80, input_dim = 43,  activation ="relu")) 

# Second hidden layer

nn.add (tf.keras.layers.Dense (units = 30, activation = "relu")) 

# Output layer

nn.add (tf.keras.layers.Dense (units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

2023-01-10 06:05:34.360350: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-10 06:05:34.368442: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-10 06:05:34.368504: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-f12233): /proc/driver/nvidia/version does not exist
2023-01-10 06:05:34.369305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model

nn.compile (loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [16]:
# Import Model Checkpoint to save the weights

from keras.callbacks import ModelCheckpoint
import os

checkpoint_path = "Resources/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname (checkpoint_path)
model_callback = tf.keras.callbacks.ModelCheckpoint (checkpoint_path, verbose =1, save_weights_only = True,period=5)

In [17]:
nn.save_weights(checkpoint_path.format (epoch = 0))

In [18]:
from keras import callbacks
# Train the model

fit_model = nn.fit (X_train_scaled, y_train, epochs=100, callbacks = [model_callback])

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5734 - accuracy: 0.7181
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5543 - accuracy: 0.7316
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7320
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5493 - accuracy: 0.7330
Epoch 5/100
789/804 [============================>.] - ETA: 0s - loss: 0.5487 - accuracy: 0.7325
Epoch 5: saving model to Resources/cp-0005.ckpt
804/804 [==============================] - 1s 2ms/step - loss: 0.5486 - accuracy: 0.7329
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7341
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5458 - accuracy: 0.7360
Epoch 9/100
804/804 [==============================] - 1s 2

In [19]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5612 - accuracy: 0.7241 - 447ms/epoch - 2ms/step
Loss: 0.5612435340881348, Accuracy: 0.7240816354751587


In [20]:
# Export our model to HDF5 file

nn.save("Resources/AlphabetSoupCharity_Optimisation.h5")

### Optimise the Model

In [21]:
# Increase more bin in CLASSIFICATION value counts >6

application_reduced = application_df. drop (['STATUS', 'USE_CASE'], axis =1)
application_reduced

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,0,N,5000,1
1,T3,Independent,C2000,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Trust,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,Association,0,N,5000,0
34295,T4,CompanySponsored,C3000,Association,0,N,5000,0
34296,T3,CompanySponsored,C2000,Association,0,N,5000,0
34297,T5,Independent,C3000,Association,0,N,5000,1


In [22]:
# Convert categorical data to numeric with `pd.get_dummies`

application_convert_reduced = pd.get_dummies(application_reduced)
application_convert_reduced

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,5000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,5000,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [23]:
# Split our preprocessed data into our features and target arrays

y_reduce = application_convert_reduced["IS_SUCCESSFUL"]
X_application_reduced = application_convert_reduced.drop(["IS_SUCCESSFUL"], axis=1)

# Split the preprocessed data into a training and testing dataset

X_train_reduced, X_test_reduced, y_train_reduced, y_test_reduced = train_test_split(X_application_reduced, y_reduce, random_state = 42, stratify =y) 

In [24]:
# Create a StandardScaler instances
scaler_reduce = StandardScaler()

# Fit the StandardScaler
X_scaler_reduce = scaler_reduce.fit(X_train_reduced)

# Scale the data
X_train_scaled_reduce = X_scaler_reduce.transform(X_train_reduced)
X_test_scaled_reduce = X_scaler_reduce.transform(X_test_reduced)

In [25]:
# Checking the number of feature:

number_feature_reduce = X_train_reduced.shape[1]
number_feature_reduce

37

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_reduce = tf.keras.models.Sequential()

# First hidden layer

nn_reduce.add(tf.keras.layers.Dense (units = 100, input_dim = 37,  activation ="tanh")) 

# Second hidden layer

nn_reduce.add (tf.keras.layers.Dense (units = 50, activation = "tanh")) 

# Third hidden layer

nn_reduce.add (tf.keras.layers.Dense (units = 20, activation = "tanh")) 

# Output layer

nn_reduce.add (tf.keras.layers.Dense (units = 1, activation = "sigmoid"))

# Check the structure of the model
nn_reduce.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               3800      
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 20)                1020      
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 9,891
Trainable params: 9,891
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model

nn_reduce.compile (loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [28]:
# Train the model

fit_model_reduce = nn_reduce.fit (X_train_scaled_reduce, y_train_reduced, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5697 - accuracy: 0.7221
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7302
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5540 - accuracy: 0.7305
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5529 - accuracy: 0.7306
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5517 - accuracy: 0.7306
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5507 - accuracy: 0.7334
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7338
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7334
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7343
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7330

In [29]:
# Evaluate the model using the test data

model_loss_reduce, model_accuracy_reduce = nn_reduce.evaluate(X_test_scaled_reduce,y_test_reduced,verbose=2)
print(f"Loss: {model_loss_reduce}, Accuracy: {model_accuracy_reduce}")

268/268 - 1s - loss: 0.5609 - accuracy: 0.7227 - 575ms/epoch - 2ms/step
Loss: 0.5608599185943604, Accuracy: 0.7226822376251221


### Changing testing method to ReLu

In [30]:
# Split the preprocessed data into a training and testing dataset

X_train_relu, X_test_relu, y_train_relu, y_test_relu = train_test_split(X_application_reduced, y_reduce, random_state = 42, stratify =y) 

In [31]:
# Create a StandardScaler instances
scaler_relu = StandardScaler()

# Fit the StandardScaler
X_scaler_relu = scaler_relu.fit(X_train_relu)

# Scale the data
X_train_scaled_relu = X_scaler_relu.transform(X_train_relu)
X_test_scaled_relu = X_scaler_relu.transform(X_test_relu)

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_relu = tf.keras.models.Sequential()

# First hidden layer

nn_relu.add(tf.keras.layers.Dense (units = 100, input_dim = 37,  activation ="relu")) 

# Second hidden layer

nn_relu.add (tf.keras.layers.Dense (units = 50, activation = "relu")) 

# Third hidden layer

nn_relu.add (tf.keras.layers.Dense (units = 20, activation = "relu")) 

# Output layer

nn_relu.add (tf.keras.layers.Dense (units = 1, activation = "sigmoid"))

# Check the structure of the model
nn_relu.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               3800      
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 20)                1020      
                                                                 
 dense_10 (Dense)            (None, 1)                 21        
                                                                 
Total params: 9,891
Trainable params: 9,891
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model

nn_relu.compile (loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [34]:
# Train the model

fit_model_relu = nn_relu.fit (X_train_scaled_relu, y_train_relu, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5694 - accuracy: 0.7227
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5558 - accuracy: 0.7306
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7329
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7341
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5507 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5506 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7336
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7334
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5494 - accura

In [35]:
# Evaluate the model using the test data

model_loss_relu, model_accuracy_relu = nn_relu.evaluate(X_test_scaled_relu,y_test_relu,verbose=2)
print(f"Loss: {model_loss_relu}, Accuracy: {model_accuracy_relu}")

268/268 - 0s - loss: 0.5740 - accuracy: 0.7236 - 449ms/epoch - 2ms/step
Loss: 0.5739560127258301, Accuracy: 0.7236151695251465


After reduced the number of features (from 43 to 37), testing with different models, increasing number of layers as well as changing the number of neurons, epochs, the accuracy percentage didn't improve much. It can be concluded that the data may have too much noise. 

Next step, we will let the computer to run and automatically choose the best model to compare with the above methods

## Checking the model against the computer's auto selection model

In [36]:
# Install keras_tuner
!pip install keras_tuner

In [37]:
# Import dependencies

import sklearn as skl
import keras_tuner as kt

In [38]:
# Use sklearn to split dataset

from sklearn.model_selection import train_test_split
X_train_sk, X_test_sk, y_train_sk, y_test_sk = train_test_split (X_application_convert, y, random_state = 42, stratify =y)

In [39]:
# Create scaler instance

X_scaler_sk = skl.preprocessing.StandardScaler()

# Fit the scaler

X_scaler_sk.fit (X_train_sk)

# Scale the data

X_train_scaled_sk = X_scaler_sk. transform(X_train_sk)
X_test_scaled_sk = X_scaler_sk.transform (X_test_sk)

In [40]:
# Create a method that creates a new Sequential model with hyperparameter options

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

# Allow kerastuner to decide which activation function to use in hidden layers

    activation = hp. Choice('activation',['relu','tanh', 'sigmoid'])

# Allow kerastuner to decide number of neurons in first layer

    nn_model.add(tf.keras.layers.Dense(units = hp.Int ('first_units', min_value = 1, max_value = 100, step=2), activation = activation, input_dim = 43))

# Allow kerastuner to decide number of hidden layers and neurons in hidden layers

    for i in range (hp.Int ('num_layers', 1,6)):
        nn_model.add (tf.keras.layers.Dense(units = hp. Int('units_' + str(i), min_value = 1, max_value = 100, step=2), activation = activation))

    nn_model.add (tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Compile the model

    nn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics= ["accuracy"])

    return nn_model

In [41]:
# Applying Hyperband

tuner = kt.Hyperband (create_model, objective= "val_accuracy", max_epochs= 100, hyperband_iterations= 2)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [42]:
# Run the kerastuner seach for best hyperparameters
tuner.search(X_train_scaled_sk, y_train_sk, epochs =50, validation_data = (X_test_scaled_sk, y_test_sk))

Trial 508 Complete [00h 03m 18s]
val_accuracy: 0.7261807322502136

Best val_accuracy So Far: 0.7285131216049194
Total elapsed time: 00h 46m 16s
INFO:tensorflow:Oracle triggered exit


In [43]:
# Get best model hyperparmeters

best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 11,
 'num_layers': 4,
 'units_0': 59,
 'units_1': 95,
 'units_2': 47,
 'units_3': 81,
 'units_4': 11,
 'units_5': 45,
 'tuner/epochs': 34,
 'tuner/initial_epoch': 12,
 'tuner/bracket': 3,
 'tuner/round': 2,
 'tuner/trial_id': '0195'}

In [44]:
# Evaluate best model against full test data 

best_model = tuner.get_best_models(1)[0]
model_loss_sk, model_accuracy_sk = best_model.evaluate (X_test_scaled_sk, y_test_sk, verbose =2)
print (f"Loss: {model_loss_sk}, Accuracy: {model_accuracy_sk}")

268/268 - 1s - loss: 0.5540 - accuracy: 0.7285 - 673ms/epoch - 3ms/step
Loss: 0.5539936423301697, Accuracy: 0.7285131216049194


#### Comment: Comparing the best model generated by a computer and the manual defined model, both models gave the similar answer : "relu" function is the best function to fit in the model with the accuracy of 72%, lost % between 55.6% and 56.1%. Besides, both models also showed 2 hidden layers were the best solution 